# Pandas Quiz AutoGrader

Upload your completed **Pandas DataFrames Quiz** notebook and get instant feedback.

## 1) Upload your completed `Pandas DataFrames Quiz` notebook

In [ ]:
#@title Choose your notebook:
from google.colab import files
uploaded = files.upload()
nb_name = list(uploaded.keys())[0]
print("Uploaded:", nb_name)


## 2) Extract quiz answers

In [ ]:
#@title Found quizzes:
import json, re, ast

with open(nb_name, "r", encoding="utf-8") as f:
    student_nb = json.load(f)

def extract_answer_dict(code_source: str):
    """Extract answers dict from a code cell that contains: answers = {...}"""
    m = re.search(r"answers\s*=\s*(\{[\s\S]*?\})\s*\n", code_source)
    if not m:
        return None
    return ast.literal_eval(m.group(1))

student_quiz_answers = {}  # {quiz_title: {Q1:'A',...}}

for cell in student_nb.get("cells", []):
    if cell.get("cell_type") != "code":
        continue
    src = cell.get("source", "")
    if isinstance(src, list):
        src = "".join(src)
    if not isinstance(src, str):
        continue
    if src.lstrip().startswith("# Your answers for:"):
        first_line = src.strip().splitlines()[0]
        qtitle = first_line.split(":", 1)[1].strip() if ":" in first_line else "Quiz"
        ans = extract_answer_dict(src)
        if ans is not None:
            student_quiz_answers[qtitle] = ans

print("Found quizzes:", list(student_quiz_answers.keys()))
student_quiz_answers


## 3) Grade and get feedback

In [ ]:
#@title Your score:
import json

answer_keys = json.loads('''{
  "Pandas DataFrames Quiz": {
    "Q1": "C",
    "Q2": "B",
    "Q3": "D",
    "Q4": "D",
    "Q5": "A"
  }
}''')

def grade_quiz(student_answers, key):
    score = 0
    missed = []
    for q, correct in key.items():
        ans = (student_answers.get(q, "") or "").strip().upper()
        if ans == correct:
            score += 1
        else:
            missed.append(f"{q} (you: {ans or 'blank'}, correct: {correct})")
    return score, len(key), missed

quiz_title = "Pandas DataFrames Quiz"

if quiz_title not in student_quiz_answers:
    print("Couldn't find the quiz answers for:", quiz_title)
    print("Make sure your notebook has a code cell that starts with:")
    print("# Your answers for:", quiz_title)
else:
    student = student_quiz_answers[quiz_title]
    key = answer_keys[quiz_title]
    score, total, missed = grade_quiz(student, key)
    print(f"=== {quiz_title} ===")
    print(f"Score: {score}/{total}")
    if missed:
        print("Missed:")
        for m in missed:
            print("-", m)
    else:
        print("Perfect! ✅")
